In [5]:
import json
import numpy
import pandas
from datetime import datetime
from pprint import pprint
import re
pandas.options.display.max_colwidth = -1
print('done')

done


In [6]:
# runs in about 5 minutes
#with open('events.json','r') as f:
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')
print('done')
#allgames_df

ok
done


In [7]:
#pdf: TOR201604250
#bdf: TOR201604250
#ldf: OAK201309040
#mygames = ['SLA192708270']

#mygames = ['OAK201309040', 'TOR201604250']

#mygames = ['BAL201208060', 'SEA201208150', 'BAL201208300', 'BAL201209300', 'BAL201304210', 'BAL201305170']

#mygames = ['BSN192704140']



mygames = ['BAL201208060']
#mygames += ['BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
#mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

somecolumns = allgames_df.columns

allgames2_df = pandas.DataFrame(columns=somecolumns)
allgames2_df['id'] = ''
allgames2_df = allgames2_df.set_index('id')

for thisid, thisrow in allgames_df.iterrows():
    if (thisid in mygames):
        allgames2_df = allgames2_df.append(thisrow)

allgames_df = allgames2_df

print('done')
#allgames_df

done


In [8]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_batting_order', 'sub_position']

player_columns = ['playerID', 'name', 'batting_order', 'position']

display_order = ['gameID', 'date', 'event_in_game', 'visiting_team', 'home_team', 'inning', 'home_or_visitor']
display_order += ['visitor_score', 'home_score', 'batterID', 'batter_hand', 'pitcherID', 'pitcher_hand', 'abflag']
display_order += ['catcherID', '1bID', '2bID', '3bID', 'ssID', 'lfID', 'cfID', 'rfID']
display_order += ['outs', 'balls', 'strikes', 'pitch_sequence']
display_order += ['runner1b', 'runner2b', 'runner3b']
display_order += ['play', 'modifiers', 'baserunning', 'comment']
print('got columns')

got columns


In [35]:
def load_rosters():
    all_rosters_df = pandas.read_csv('fullroster.csv')
    all_rosters_df['year'] = pandas.to_numeric(all_rosters_df['year'], errors='coerce')
    return all_rosters_df

def get_game_info(onerow, theindex):

    info_df = pandas.DataFrame.from_records(onerow['info'], index=theindex)

    teams_df = pandas.read_csv('teams.csv')
       
    info_df['visleague'] = teams_df['league'][teams_df['teamID'] == info_df['visteam'].tolist()[0]].tolist()[0][0:1]
    info_df['homeleague'] = teams_df['league'][teams_df['teamID'] == info_df['hometeam'].tolist()[0]].tolist()[0][0:1]

    info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
    info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
    info_df['dayofweek'] = info_df['date'].apply(lambda x: datetime.strftime(x, '%a'))
    
    info_df['gameno'] = pandas.to_numeric(info_df['number'], errors='coerce')
    info_df = info_df.drop('number', axis=1)
    info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
    info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
    info_df['timeofgame'] = pandas.to_numeric(info_df['timeofgame'], errors='coerce')
    info_df = info_df.fillna('')

    return info_df


def get_starters(onerow):
    vsdf = pandas.DataFrame(onerow['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onerow['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['batting_order'] = pandas.to_numeric(df['batting_order'])
    df['position'] = pandas.to_numeric(df['position'])

    return df


def get_inning_info(thisevent, infodf, inningno):
    
    thisevent['visiting_team'] = infodf['visteam'].tolist()[0]
    thisevent['home_team'] = infodf['hometeam'].tolist()[0]
    thisevent['inning'] = inningno
    if (side == 0):
        thisevent['batting_team'] = thisevent['visiting_team']
        thisevent['fielding_team'] = thisevent['home_team']
    else:
        thisevent['batting_team'] = thisevent['home_team']
        thisevent['fielding_team'] = thisevent['visiting_team']
    return thisevent

   
def get_fielders(thisevent, playing_df, fielding_flag):
                    
    thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 1)].tolist()[0]                               

    
    if (re.search('padj: ', thisevent['comment'].lower())):   # correct if there is a badj
        thisevent['pitcher_hand'] = thisevent['comment'][re.search('padj: ', thisevent['comment'].lower()).end():re.search('padj: ', thisevent['comment'].lower()).end()+1]
    else:
        thisevent['pitcher_hand'] = roster_df['throwing_hand'][
            (roster_df['playerID'] == thisevent['pitcherID']) & 
            (roster_df['year'] == float(thisevent['gameID'][3:7])) 
            ].tolist()[0]
    
    thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 2)].tolist()[0]
    thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 3)].tolist()[0]
    thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 4)].tolist()[0]
    thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 5)].tolist()[0]
    thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 6)].tolist()[0]
    thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 7)].tolist()[0]
    thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 8)].tolist()[0]
    thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 9)].tolist()[0]

    return thisevent


def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['play'] = df['play'].str.split('.').str.get(0)
    
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')

    df['baserunning'] = df['event'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df


def get_sub_name(players_curr, theroster, evnt, gameinfo):
    
    if (evnt['sub_team'] == 0):
        sub_team_name = gameinfo['visteam'].values[0]
    else:
        sub_team_name = gameinfo['hometeam'].values[0]
    yeer = gameinfo['date'].apply(lambda x: x.year).values[0]
    sub_name_first = theroster['nameFirst'].iloc[
            numpy.where(
                (theroster['playerID'] == evnt['batterID']) & 
                (theroster['year'] == yeer) &
                (theroster['team'] == sub_team_name)
            )
        ].values[0]
    sub_name_last = theroster['nameLast'].iloc[
            numpy.where(
                (theroster['playerID'] == evnt['batterID']) & 
                (theroster['year'] == yeer) &
                (theroster['team'] == sub_team_name)
            )
        ].values[0]
    sub_name = '"' + sub_name_first + ' ' + sub_name_last + '"'

    return sub_name

def make_substitution(players_curr, theroster, evnt, gameinfo, basers):
#    print('\n')
#    print('SUBSTITUTING...')
    if (evnt['home_or_visitor'] == evnt['sub_team']):
        
        if (evnt['sub_position'] == 12):
#            print('pinch runner')
            old_runner = players_curr['playerID'].loc[
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
            ].values[0]
            new_runner = evnt['batterID']
#            print(old_runner, new_runner)
            for i in range(0,len(basers)):
                if (basers[i] == old_runner):
                    basers[i] = new_runner
#            print(basers)
        
        players_curr.loc[
            (players_curr['which_team'] == evnt['sub_team']) & 
            (players_curr['batting_order'] == evnt['sub_batting_order'])
            , 'playerID'] = evnt['batterID']

        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
                ), 'name'] = get_sub_name(players_curr, theroster, evnt, gameinfo)
        
    else:
        
        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
                ), 'playerID'] = evnt['batterID']
        
        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
                ), 'name'] = get_sub_name(players_curr, theroster, evnt, gameinfo)
#        print('New lineup:\n')
#    print(players_curr)
    return (players_curr, basers)

def advance_runners(evnt, basers, oots, scr, teem, debug):   # If there is baserunning info provided, obey it
    
    # If there are multiple baserunning events, split them around ;
    x = evnt['baserunning'].split(';')

    success = []
    failure = []

    for i in range(0, len(x)):
        if ('-' in x[i]):
            success.append([x[i].split('-')[0], x[i].split('-')[1][0], x[i].split('-')[1][1:]])
        if ('X' in x[i]):
            failure.append([x[i].split('X')[0], x[i].split('X')[1][0], x[i].split('X')[1][1:]])

    for u in (success, failure):
        for j in range(0, len(u)):
            if (u[j][0] == 'B'):
                u[j][0] = 0
            if (u[j][1] == 'H'):
                u[j][1] = 4
            for k in range(0,2):
                u[j][k] = int(u[j][k])

#    print('Failure matrix: ', failure)
#    print('Success matrix: ', success)
    
    for y in range(0, len(failure)):
        oots = oots + 1
        basers[failure[y][0]-1] = ''
        if (failure[y][1] < 4):            # no need to clear a base if runner is out at home
            basers[failure[y][1]-1] = ''
        evnt['comment'] += failure[y][2]     

    for z in range(0, len(success)):
        if (success[z][1] == 4):
            scr[teem] = scr[teem] + 1
        elif (success[z][0] == 0):
            basers[success[z][1]-1] = evnt['batterID']
        else:
            basers[success[z][1]-1] = basers[success[z][0]-1]

        basers[success[z][0]-1] = ''   # regardless, clear the base he was originally on
        evnt['comment'] += success[z][2]
    
    return (evnt, basers, oots, scr)


def advance_batter(evnt, basers, oots, scr, teem, debug):   
#    print('\n')    
#    print('Advancing runners, then batter: Inning {0:.0f}, side {1:.0f}'.format(evnt['inning'], evnt['home_or_visitor']))
   
    breakout = False
    if (debug == 2):
        print('In advance batter function; breakout is starting as ',breakout)
    
    if (evnt['play'][0] == 'N' and (not(breakout))):  # No play means no at-bat and break out of if series
        if (debug == 2):
            print('no play')
        evnt['abflag'] = 0
        breakout = True
    
    elif (re.match(r'WP|PB|SB|CS|DI', evnt['play']) and (not(breakout))): # baserunning events not involving the batter
        if (debug == 2):
            print('baserunning event not involving batter')

        evnt['abflag'] = 0
        if (re.match(r'SB', evnt['play'])):   # advance base-stealer
            if (debug == 2):
                print('Stolen base')
            stolen_base = evnt['play'][2]
            if (stolen_base == 'H'):
                stolen_base = 4
            else:
                stolen_base = int(stolen_base)              
            orig_base = stolen_base - 1            
            if (stolen_base < 4):   # only move ahead one base if home not stolen
                basers[stolen_base-1] = basers[orig_base-1]
            basers[orig_base-1] = ''      
            
        if (re.match(r'CS', evnt['play'])):   # drop caught-stealer and increment outs
            if (debug == 2):
                print('Caught stealing')
            caught_base = evnt['play'][2]
            if (caught_base == 'H'):
                caught_base = 4
            else:
                caught_base = int(caught_base)
            orig_base = caught_base - 1
            oots = oots + 1
            basers[caught_base-1] = ''
        
        breakout = True
        
    elif ((re.match(r'W|I|C|HP',evnt['play'])) and (not(breakout))):   # batter to first but no at-bat
        if (debug == 2):
            print('Walk or similar')
            
        thisevent['abflag'] = 0  # not an at-bat
        if ('B' not in evnt['baserunning']):     # if explicit batter advance not provided...
            basers[0] = evnt['batterID']              # advance batter to first
        breakout = True
    
    
    elif ((evnt['play'][0] in ('S', 'F', 'E')) and (not(breakout))):    # single/FC/error: batter to first with at-bat
        if (debug == 2):
            print('Single or similar')
                    
        if ('B' not in evnt['baserunning']):       # (unless batter advance provided explicitly earlier)
            basers[0] = evnt['batterID']
        breakout = True

        
    elif ((evnt['play'][0] == 'D') and (not(breakout))):
        if (debug == 2):
            print('Double or similar')

        if ('B' not in evnt['baserunning']):       # (unless batter advance provided explicitly earlier)
            basers[1] = evnt['batterID']
        breakout = True

    elif ((evnt['play'][0] == 'T') and (not(breakout))):
        if (debug == 2):
            print('Triple or similar')
        
        if ('B' not in evnt['baserunning']):       # (unless batter advance provided explicitly earlier)
            basers[2] = evnt['batterID']
        breakout = True

    elif ((re.match(r'HR', evnt['play'])) and (not(breakout))):   # home run
        if (debug == 2):
            print('Home run')

#            print('Home run!')
#            print('Score was ',scr)
            scr[teem] = scr[teem] + 1            # add 1 to score for batter crossing home
            basers = ['', '', '']                   # empty bases since everyone has scored
            
    else:
        print('wtf starting letter not in set!')
        print(evnt['play'])

#    print ('Now on base: [{0}, {1}, {2}] with {3} outs; score {4}-{5}'.format(basers[0], basers[1], basers[2], oots, scr[0], scr[1]))
#    print('\n')

    if (debug == 2):
        print('\n')
    
    return (evnt, basers, oots, scr)

print('done')

done


In [36]:
debug = 2   # 0: no debug output ; 1: output in main body of code; 2: output also in functions

if (debug == 2):
    print('debug = {:}'.format(debug))
nGames = len(allgames_df)
cnt = 0

roster_df = load_rosters()

events_df = pandas.DataFrame()

for thisgameID,thisrow in allgames_df.iterrows():
    cnt = cnt + 1
#    if (numpy.mod(cnt,10) == 0):
    print('Processing game {0:,.0f} of {1:,.0f}: {2}...'.format(cnt, nGames, thisgameID))
    game_events_df = pandas.DataFrame()
        
    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
    thisgame_df = thisgame_df.fillna('')
    thisgame_df['gameID'] = thisgameID
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df['sub_team'] = pandas.to_numeric(thisgame_df['sub_team'])
    thisgame_df['sub_batting_order'] = pandas.to_numeric(thisgame_df['sub_batting_order'])
    thisgame_df['sub_position'] = pandas.to_numeric(thisgame_df['sub_position'])
    
    
    thisgame_df = thisgame_df.set_index('event_in_game')
    
    this_info_df = get_game_info(thisrow, allgames_df.index) # get constants of game info
    playing_df = get_starters(thisrow) # get starters
    thisgame_df = split_event(thisgame_df) # split all events in game into parts    
    score = [0,0] # score as [vis, home] starts at [0,0]

    foundinnings = thisgame_df['inning'].drop_duplicates().values.tolist()
    theseinnings = numpy.arange(1, numpy.max(foundinnings)+1) # it's +2 b/c it expects a 0-based list

    for ing in theseinnings:
        for side in range(0,2):                           # iter through each half-inning
            if (debug > 0):
                print('Inning {0:.0f}, side {1:.0f}'.format(ing, side))
            halfinn = thisgame_df[(thisgame_df['inning'] == ing) & (thisgame_df['home_or_visitor'] == side)]
            onbase = ['', '', '']
            nOuts = 0            #     and there are no outs
            fieldingflag = 1 - side  # flag: makes fielding team the opposite of batting team

            for eig, thisevent in halfinn.iterrows():    # iter through all events in this half-inn    
                    
                if (thisevent['event_type'] == 'sub'):
                    thisyear = this_info_df['date'].apply(lambda x: x.year)
                    (playing_df, onbase) = make_substitution(playing_df, roster_df, thisevent, this_info_df, onbase)
                else:
                    if (debug > 0):
                        print(thisevent[['inning', 'home_or_visitor', 'batterID', 'event_type', 'event', 'comment']].tolist())
#                        print(onbase)
#                        print('\n')
                    thisevent['visiting_team'] = this_info_df['visteam'].tolist()[0]
                    thisevent['home_team'] = this_info_df['hometeam'].tolist()[0]
                    thisevent['date'] = this_info_df['date'].tolist()[0]
                    thisevent['inning'] = ing
                    if (side == 0):
                        thisevent['batting_team'] = thisevent['visiting_team']
                        thisevent['fielding_team'] = thisevent['home_team']
                    else:
                        thisevent['batting_team'] = thisevent['home_team']
                        thisevent['fielding_team'] = thisevent['visiting_team']

                    thisevent = get_fielders(thisevent, playing_df, fieldingflag) # get fielders, including pitcher

                    thisevent['abflag'] = 1
                    thisevent['balls'] = pandas.to_numeric(thisevent['pitch_count'][0:1], errors='coerce')
                    thisevent['strikes'] = pandas.to_numeric(thisevent['pitch_count'][1:2], errors='coerce')

                    # batterID gets loaded from raw data, but look up his handendess... and....
                    # DON'T FORGET TO CORRECT THIS FOR BADJ IF NEEDED!
                    
                    if (re.search('badj: ', thisevent['comment'].lower())):   # correct if there is a badj
                        thisevent['batter_hand'] = thisevent['comment'][re.search('badj: ', thisevent['comment'].lower()).end():re.search('badj: ', thisevent['comment'].lower()).end()+1]
                    else:
                        thisevent['batter_hand'] = roster_df['batting_hand'][(roster_df['playerID'] == thisevent['batterID']) & (roster_df['team'] == thisevent['batting_team']) & (roster_df['year'] == float(thisevent['gameID'][3:7]))].tolist()[0]

                    # if there is baserunning info, implement it first
                    if (thisevent['baserunning'] != ''):
                        (thisevent, onbase, nOuts, score) = advance_runners(
                            thisevent,
                            onbase,
                            nOuts,
                            score,
                            side,
                            debug)               
                        
#                    if (debug > 0):
#                        print('entering advance batter section on play number {:}'.format(eig))
                        
                    # If the play starts with fielder info or is a strikeout, increment outs
                    if (re.match(r'[0-9K]+',thisevent['play'])):
#                        if (debug > 0):
#                            print('out')
                        nOuts = nOuts + 1
                    else:
#                        if (debug > 0):
#                            print('not out')
                    # If the play is a hit, advance baserunners with fcn advance_runners(evnt, basers, oots, scr, teem):
                        (thisevent, onbase, nOuts, score) = advance_batter(
                            thisevent,
                            onbase,
                            nOuts,
                            score,
                            side,
                            debug)

                    # update info based on the play that has occurred
                    thisevent['outs'] = nOuts
                    thisevent['runner1b'] = onbase[0]
                    thisevent['runner2b'] = onbase[1]
                    thisevent['runner3b'] = onbase[2]
                    thisevent['visitor_score'] = score[0]
                    thisevent['home_score'] = score[1]

                # finally, append this play to the list of plays
                game_events_df = game_events_df.append(thisevent)
            if (debug > 0):
                print('\n')
                
    game_events_df = game_events_df[game_events_df['event_type'] == 'play']
    game_events_df = game_events_df[game_events_df['play'] != 'NP']

    game_events_df = game_events_df.reset_index()
    game_events_df.index.name = 'event_in_game'
    
    events_df = pandas.concat((events_df, game_events_df), axis=0)

#events_df = events_df.fillna('')

events_df = events_df.reset_index()

events_df['event_in_game'] = events_df['event_in_game'] + 1  # make it a 1-based counter
events_df['date'] = pandas.to_datetime(events_df['date'], errors='coerce')

events_df.index.name = 'eventID'

events_df.sort_values('gameID')

display_order = ['gameID', 'event_in_game' 'inning', 'home_or_visitor']
display_order += ['batterID', 'batter_hand', 'pitcherID', 'pitcher_hand']
display_order += ['play', 'modifiers', 'baserunning', 'comment']

#print('Writing file...')
#events_df.to_csv('events_2012-2016.csv', encoding='utf-8')
#print('Done!')

print('Final score: ',score)
#events_df = events_df[display_order]
#events_df

debug = 2
Processing game 1 of 1: BAL201208060...
Inning 1, side 0
[1, 0, 'ackld001', 'play', 'K23', '']
[1, 0, 'saunm001', 'play', '8/F', '']
[1, 0, 'montj003', 'play', '13/G-', '']


Inning 1, side 1
[1, 1, 'markn001', 'play', 'S8/G', '']
In advance batter function; breakout is starting as  False
Single or similar


[1, 1, 'andir001', 'play', 'K', '']
[1, 1, 'hardj003', 'play', '2/P2F', '']
[1, 1, 'jonea003', 'play', 'D8/L+.1-3', '']
In advance batter function; breakout is starting as  False
Double or similar


[1, 1, 'wietm001', 'play', '1/G-', '']


Inning 2, side 0
[2, 0, 'jasoj001', 'play', '8/F', '']
[2, 0, 'seagk001', 'play', 'K', '']
[2, 0, 'carpm001', 'play', '7/L', '']


Inning 2, side 1
[2, 1, 'davic003', 'play', 'K', '']
[2, 1, 'fordl001', 'play', 'S7/L', '']
In advance batter function; breakout is starting as  False
Single or similar


[2, 1, 'reynm001', 'play', 'D8/L.1-H', '']
In advance batter function; breakout is starting as  False
Double or similar


[2, 1, 'quino001